In [226]:
import os
import glob

from typing import Dict, List
from pathlib import Path
from collections import Counter

In [209]:
#unlabelled
with open("../data/results/unlabelled_sts_bbox_metadata.txt", 'r') as f:
    lines = f.readlines()
    data = [line.replace("\n", "").split(" ") for line in lines]
    #print(lines[:10])
    img_bbox_data = [{"filename": row[0], "frame":row[1], "bbox_idx": int(row[-1])} for row in data]    

with open("../data/results/secondary_predictions.txt", 'r') as f:
    lines = f.readlines()
    data = [line.replace("\n", "").split(" ") for line in lines]
    bbox_preds = {row[0]: {"label": " ".join(row[1:-1]), "conf": float(row[-1])} for row in data}
    print(bbox_preds)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [210]:
image_frame_dict = dict()
for frame in img_bbox_data:
    filename = frame["filename"]
    frame_name = frame["frame"]
    bbox_idx = frame["bbox_idx"]
    
    
    if image_frame_dict.get(filename) is None:
        image_frame_dict[filename] = list()
    
    image_frame_dict[filename].append((frame_name, original_label, bbox_idx))
image_frame_dict

{'1277103759Image000002.jpg': [('frame00000.jpg', 'STOP', 0),
  ('frame00001.jpg', 'STOP', 1)],
 '1277103759Image000003.jpg': [('frame00002.jpg', 'STOP', 0),
  ('frame00003.jpg', 'STOP', 1)],
 '1277103759Image000004.jpg': [('frame00004.jpg', 'STOP', 0),
  ('frame00005.jpg', 'STOP', 1)],
 '1277103759Image000005.jpg': [('frame00006.jpg', 'STOP', 0),
  ('frame00007.jpg', 'STOP', 1)],
 '1277103759Image000007.jpg': [('frame00008.jpg', 'STOP', 0),
  ('frame00009.jpg', 'STOP', 1)],
 '1277103759Image000008.jpg': [('frame00010.jpg', 'STOP', 0),
  ('frame00011.jpg', 'STOP', 1),
  ('frame00012.jpg', 'STOP', 2)],
 '1277103759Image000009.jpg': [('frame00013.jpg', 'STOP', 0),
  ('frame00014.jpg', 'STOP', 1)],
 '1277103759Image000010.jpg': [('frame00015.jpg', 'STOP', 0),
  ('frame00016.jpg', 'STOP', 1)],
 '1277103759Image000012.jpg': [('frame00017.jpg', 'STOP', 0),
  ('frame00018.jpg', 'STOP', 1)],
 '1277103759Image000013.jpg': [('frame00019.jpg', 'STOP', 0),
  ('frame00020.jpg', 'STOP', 1)],
 '12771

In [211]:
path_secondary_dataset = "../data/secondary_dataset/"
labels = os.listdir(path_secondary_dataset)
num_changed = 0
for label in labels:
    frames = os.listdir(os.path.join(path_secondary_dataset, label))
    for frame in frames:
        if bbox_preds[frame]["label"] != label:
            num_changed += 1
        bbox_preds[frame]["label"] = label
print(f"{num_changed} labels have been corrected through visual examination.")

# Remove frames from 'bbox_preds' which was not found in 'secondary_dataset'
secondary_dataset_frames = list()
bbox_preds_frames = list(bbox_preds.keys())
for label in labels:
    secondary_dataset_frames += os.listdir(os.path.join(path_secondary_dataset, label))

frames_to_remove = set(bbox_preds_frames) - set(secondary_dataset_frames)
for frame in frames_to_remove:
    
    del bbox_preds[frame]
print(f"{len(frames_to_remove)} frames has been removed")

264 labels have been corrected through visual examination.
86554 frames has been removed


In [212]:
bbox_preds

{'frame00052.jpg': {'label': 'garbage', 'conf': 0.7259234189987183},
 'frame00064.jpg': {'label': 'garbage', 'conf': 0.6220216155052185},
 'frame00065.jpg': {'label': 'garbage', 'conf': 0.643958568572998},
 'frame00066.jpg': {'label': 'garbage', 'conf': 0.6194491982460022},
 'frame00067.jpg': {'label': 'garbage', 'conf': 0.6324260234832764},
 'frame00068.jpg': {'label': 'garbage', 'conf': 0.5311448574066162},
 'frame00069.jpg': {'label': 'garbage', 'conf': 0.7954269647598267},
 'frame00070.jpg': {'label': 'garbage', 'conf': 0.6533210873603821},
 'frame00071.jpg': {'label': 'garbage', 'conf': 0.5174459218978882},
 'frame00072.jpg': {'label': 'garbage', 'conf': 0.47632965445518494},
 'frame00073.jpg': {'label': 'garbage', 'conf': 0.603390097618103},
 'frame00074.jpg': {'label': 'garbage', 'conf': 0.7017354965209961},
 'frame00075.jpg': {'label': 'garbage', 'conf': 0.5593442916870117},
 'frame00079.jpg': {'label': 'garbage', 'conf': 0.7457676529884338},
 'frame00080.jpg': {'label': 'garba

In [213]:

def autocorrect_unlabelled_based_on_secondary_dataset_folder(path_to_txt:str,
                                                             label2idx:Dict[str, int], 
                                                             ignore_list: List[str]):
    save_path = "../data/results/new_labels/"
    Path(save_path).mkdir(exist_ok=True, parents=True)
    new_labels_count = list()
    
    # Make dicts out of data
    txt_paths = glob.glob(os.path.join(path_to_txt, "*.txt"))
    
    for txt_path in tqdm(txt_paths):
        empty_newlines(txt_path)
        filename = os.path.basename(txt_path).replace(".txt", ".jpg")
        bbox_data = image_frame_dict.get(filename)
        
        # Read txt file
        with open(txt_path, 'r') as f:
            original_txt_data = f.readlines()
            txt_data = original_txt_data.copy()
 
        if bbox_data is not None:
            for bbox in bbox_data:
                
                # Original metadata
                frame_name     = bbox[0]
                original_label = bbox[1]
                bbox_idx       = bbox[2]

                # Prediction data
                if bbox_preds.get(frame_name) is None:

                    pass
                else:

                    bbox_pred = bbox_preds[frame_name]
                    prediction_label = bbox_pred["label"]
                    prediction_conf = bbox_pred["conf"]
                    

                    if prediction_label == "garbage":
                        # Throw away this bounding box
                        txt_data[bbox_idx] = "\n"
                        
                        new_labels_count.append(prediction_label)
                        
                    elif prediction_label in ignore_list:
                        pass

                    else:
                        # Replace the original label with the predicted label
                        if prediction_conf > 0.5:
                            new_labels_count.append(prediction_label)
                            txt_string = txt_data[bbox_idx]
                            txt_string = txt_string.split(" ")
                            txt_string[0] =  str(label2idx[prediction_label])
                            txt_string = " ".join(txt_string)
                            txt_data[bbox_idx] = txt_string

        if has_changed(txt_data, original_txt_data):
            with open(os.path.join(save_path, os.path.basename(txt_path)), 'w') as f:
                for line in txt_data:
                    if line != "\n":

                        f.write(line)
                        
    print(Counter(new_labels_count))
    
    
path_to_txt = "../../../Datasets/Swedish_Traffic_Signs/STS_COCO_dataset_v2_part1/STS_COCO_dataset_v2/labels/unlabelled_sts/"
label = "STOP"

label_file = "../../../../Downloads/STS_COCO.names"
with open(label_file, "r") as f:
    label2idx = {line.replace("\n", ""):idx for idx, line in enumerate(f.readlines())}
    
ignore_list = []
autocorrect_unlabelled_based_on_secondary_dataset_folder(path_to_txt, label2idx, ignore_list)


Counter({'garbage': 2870})


# Follow this path if you have done the following

- Cropped bounding boxes for each labels <cropped_img_dir>/<class>/*.jpg
- Trained a model
- Predicted a specific label e.g. '../data/unlabelled_sts/STOP' and obtained the 'secondary_predictions.txt' file

The following cells does one of the following
    
1. Use 'secondary_predictions.txt' and replace all labels in original object detection dataset
2. Use visual inspection (moving files by hand) in the 'data/secondary_dataset/<class>/*.jpg' directory  and replace all labels in original object detection dataset

In [180]:
# Labelled dataset
with open("../data/results/unlabelled_sts_bbox_metadata.txt", 'r') as f:
    lines = f.readlines()
    data = [line.replace("\n", "").split(" ") for line in lines]
    print(lines)
    img_bbox_data = [{"filename": row[0], "label": " ".join(row[1:-1]).split("\\")[0], "frame": " ".join(row[1:-1]).split("\\")[1], "bbox_idx": int(row[-1])} for row in data]    
    
with open("../data/results/secondary_predictions.txt", 'r') as f:
    lines = f.readlines()
    data = [line.replace("\n", "").split(" ") for line in lines]
    bbox_preds = {row[0]: {"label": " ".join(row[1:-1]), "conf": float(row[-1])} for row in data}
    print(img2bbox)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IndexError: list index out of range

In [179]:
img_bbox_data = [frame for frame in img_bbox_data if frame["label"] == "STOP"]
img_bbox_data

[{'filename': '1277104235Image000084.jpg',
  'label': 'STOP',
  'frame': 'frame00328.jpg',
  'bbox_idx': 5},
 {'filename': '1277104235Image000124.jpg',
  'label': 'STOP',
  'frame': 'frame00412.jpg',
  'bbox_idx': 8},
 {'filename': '1277104314Image000001.jpg',
  'label': 'STOP',
  'frame': 'frame00625.jpg',
  'bbox_idx': 0},
 {'filename': '1277104314Image000001.jpg',
  'label': 'STOP',
  'frame': 'frame00628.jpg',
  'bbox_idx': 3},
 {'filename': '1277104314Image000003.jpg',
  'label': 'STOP',
  'frame': 'frame00630.jpg',
  'bbox_idx': 1},
 {'filename': '1277104314Image000003.jpg',
  'label': 'STOP',
  'frame': 'frame00632.jpg',
  'bbox_idx': 3},
 {'filename': '1277104314Image000004.jpg',
  'label': 'STOP',
  'frame': 'frame00636.jpg',
  'bbox_idx': 3},
 {'filename': '1277104314Image000005.jpg',
  'label': 'STOP',
  'frame': 'frame00638.jpg',
  'bbox_idx': 1},
 {'filename': '1277104314Image000005.jpg',
  'label': 'STOP',
  'frame': 'frame00642.jpg',
  'bbox_idx': 5},
 {'filename': '1277

In [169]:
image_frame_dict = dict()
for frame in img_bbox_data:
    filename = frame["filename"]
    original_label = frame["label"]
    frame_name = frame["frame"]
    bbox_idx = frame["bbox_idx"]
    
    
    if image_frame_dict.get(filename) is None:
        image_frame_dict[filename] = list()
    
    image_frame_dict[filename].append((frame_name, original_label, bbox_idx))
image_frame_dict

{'1277104235Image000084.jpg': [('frame00328.jpg', 'STOP', 5)],
 '1277104235Image000124.jpg': [('frame00412.jpg', 'STOP', 8)],
 '1277104314Image000001.jpg': [('frame00625.jpg', 'STOP', 0),
  ('frame00628.jpg', 'STOP', 3)],
 '1277104314Image000003.jpg': [('frame00630.jpg', 'STOP', 1),
  ('frame00632.jpg', 'STOP', 3)],
 '1277104314Image000004.jpg': [('frame00636.jpg', 'STOP', 3)],
 '1277104314Image000005.jpg': [('frame00638.jpg', 'STOP', 1),
  ('frame00642.jpg', 'STOP', 5)],
 '1277104314Image000006.jpg': [('frame00647.jpg', 'STOP', 4)],
 '1277104314Image000008.jpg': [('frame00651.jpg', 'STOP', 3),
  ('frame00652.jpg', 'STOP', 4)],
 '1277104314Image000009.jpg': [('frame00656.jpg', 'STOP', 3)],
 '1277104314Image000010.jpg': [('frame00657.jpg', 'STOP', 0),
  ('frame00660.jpg', 'STOP', 3)],
 '1277104314Image000011.jpg': [('frame00661.jpg', 'STOP', 0),
  ('frame00665.jpg', 'STOP', 4)],
 '1277104314Image000013.jpg': [('frame00667.jpg', 'STOP', 1),
  ('frame00670.jpg', 'STOP', 4)],
 '1277104314I

In [170]:
# Does all frames have an original .jpg?
for frame in bbox_preds:
    exist = False
    for img in image_frame_dict:
        for bbox in image_frame_dict[img]:
            if bbox[0] == frame:
                exist = True
    assert exist

# Alternative 1: Append predictions above certain threshold to dataset

In [153]:
# Filter all below certain conf thresholds
bbox_preds = {frame:bbox_preds[frame] for frame in bbox_preds if bbox_preds[frame]["conf"] >= 0.979}
print(Counter([bbox_preds[frame]["label"] for frame in bbox_preds]))

Counter({'STOP': 525, 'OTHER': 90, 'truck': 11, '50_SIGN': 5, 'car': 3, '80_SIGN': 2})


In [144]:


# Clean up all '\n'    
def empty_newlines(txt):
    with open(txt, 'r') as f:
        lines = f.readlines()
        a = len(lines)
        lines = [line for line in lines if line != "\n"]
        b = len(lines)
        if a-b != 0:
            with open(txt, 'w') as f:
                for line in lines:
                    f.write(line)

def has_changed(arr1, arr2):
    
    if len(arr1) != len(arr2):
        return True
    
    same = True
    for a, b in zip(arr1, arr2):
        if a != b:
            same = False
    if same:
        return False
    else:
        return True

def autocorrect_one_label(path_to_txt:str, label: str, label2idx:Dict[str, int], ignore_list: List[str]):
    save_path = "../data/results/new_labels/"
    Path(save_path).mkdir(exist_ok=True, parents=True)
    new_labels_count = list()
    
    # Make dicts out of data
    txt_paths = glob.glob(os.path.join(path_to_txt, "*.txt"))
    
    for txt_path in tqdm(txt_paths):
        empty_newlines(txt_path)
        filename = os.path.basename(txt_path).replace(".txt", ".jpg")
        bbox_data = image_frame_dict.get(filename)
        
        # Read txt file
        with open(txt_path, 'r') as f:
            original_txt_data = f.readlines()
            txt_data = original_txt_data.copy()
 
        if bbox_data is not None:
            for bbox in bbox_data:
                
                # Original metadata
                frame_name     = bbox[0]
                original_label = bbox[1]
                bbox_idx       = bbox[2]
                


                # Prediction data
                if bbox_preds.get(frame_name) is None:

                    pass
                else:

                    bbox_pred = bbox_preds[frame_name]
                    prediction_label = bbox_pred["label"]

                    new_labels_count.append(prediction_label)
                    #prediction_conf  = bbox_pred["conf"]

                    #print(prediction_label)
                    if prediction_label == "garbage":
                        # Throw away this bounding box
                        txt_data[bbox_idx] = "\n"
                        
                    elif prediction_label in ignore_list:
                        pass

                    else:
                        if prediction_label != "STOP":
                            # Replace the original label with the predicted label
                            txt_string = txt_data[bbox_idx]
                            txt_string = txt_string.split(" ")
                            txt_string[0] =  str(label2idx[prediction_label])
                            txt_string = " ".join(txt_string)
                            txt_data[bbox_idx] = txt_string
                            
        if has_changed(txt_data, original_txt_data):
            with open(os.path.join(save_path, os.path.basename(txt_path)), 'w') as f:
                for line in txt_data:
                    if line != "\n":
                        
                        if int(line.split(" ")[0]) > 33:
                            print(line)
                        f.write(line)
                        
    print(Counter(new_labels_count))
    
    
path_to_txt = "../../../Datasets/Swedish_Traffic_Signs/STS_COCO_dataset_v2_part1/STS_COCO_dataset_v2/labels/unlabelled_sts/"
label = "STOP"

label_file = "../../../../Downloads/STS_COCO.names"
with open(label_file, "r") as f:
    label2idx = {line.replace("\n", ""):idx for idx, line in enumerate(f.readlines())}
    
ignore_list = []
autocorrect_one_label(path_to_txt, label, label2idx, ignore_list)


Counter({'STOP': 525, 'OTHER': 448, 'PASS_RIGHT_SIDE': 19, 'NO_PARKING': 16, 'NO_STOPPING_NO_STANDING': 11, 'car': 6, 'garbage': 4, '50_SIGN': 4, '30_SIGN': 3, 'person': 2, '110_SIGN': 2, '80_SIGN': 1, 'PASS_EITHER_SIDE': 1, '120_SIGN': 1})


# Alternative 2: Visually examine and correct labels in the 'secondary_dataset' directory

Run the script 

```python 04_create_secondary_dataset.py --img-dir data\unlabelled_sts2\STOP  --threshold 0``` 

with threshold value zero

In [174]:
path_secondary_dataset = "../data/secondary_dataset/"
labels = os.listdir(path_secondary_dataset)
num_changed = 0
for label in labels:
    frames = os.listdir(os.path.join(path_secondary_dataset, label))
    for frame in frames:
        if bbox_preds[frame]["label"] != label:
            num_changed += 1
        bbox_preds[frame]["label"] = label
print(f"{num_changed} labels have been corrected through visual examination.")

# Remove frames from 'bbox_preds' which was not found in 'secondary_dataset'
secondary_dataset_frames = list()
bbox_preds_frames = list(bbox_preds.keys())
for label in labels:
    secondary_dataset_frames += os.listdir(os.path.join(path_secondary_dataset, label))

frames_to_remove = set(bbox_preds_frames) - set(secondary_dataset_frames)
for frame in frames_to_remove:
    print(frame)
print(f"{len(frames_to_remove)} frames has been removes")

0 labels have been corrected through visual examination.


In [176]:

def autocorrect_one_label_based_on_secondary_dataset_folder(path_to_txt:str, 
                                                            label: str, 
                                                            label2idx:Dict[str, int], 
                                                            ignore_list: List[str]):
    save_path = "../data/results/new_labels/"
    Path(save_path).mkdir(exist_ok=True, parents=True)
    new_labels_count = list()
    
    # Make dicts out of data
    txt_paths = glob.glob(os.path.join(path_to_txt, "*.txt"))
    
    for txt_path in tqdm(txt_paths):
        empty_newlines(txt_path)
        filename = os.path.basename(txt_path).replace(".txt", ".jpg")
        bbox_data = image_frame_dict.get(filename)
        
        # Read txt file
        with open(txt_path, 'r') as f:
            original_txt_data = f.readlines()
            txt_data = original_txt_data.copy()
 
        if bbox_data is not None:
            for bbox in bbox_data:
                
                # Original metadata
                frame_name     = bbox[0]
                original_label = bbox[1]
                bbox_idx       = bbox[2]

                # Prediction data
                if bbox_preds.get(frame_name) is None:

                    pass
                else:

                    bbox_pred = bbox_preds[frame_name]
                    prediction_label = bbox_pred["label"]

                    new_labels_count.append(prediction_label)

                    if prediction_label == "garbage":
                        # Throw away this bounding box
                        txt_data[bbox_idx] = "\n"
                        
                    elif prediction_label in ignore_list:
                        pass

                    else:
                        if prediction_label != "STOP":
                            # Replace the original label with the predicted label
                            txt_string = txt_data[bbox_idx]
                            txt_string = txt_string.split(" ")
                            txt_string[0] =  str(label2idx[prediction_label])
                            txt_string = " ".join(txt_string)
                            txt_data[bbox_idx] = txt_string
                            
        if has_changed(txt_data, original_txt_data):
            with open(os.path.join(save_path, os.path.basename(txt_path)), 'w') as f:
                for line in txt_data:
                    if line != "\n":
                        
                        if int(line.split(" ")[0]) > 33:
                            print(line)
                        f.write(line)
                        
    print(Counter(new_labels_count))
    
    
path_to_txt = "../../../Datasets/Swedish_Traffic_Signs/STS_COCO_dataset_v2_part1/STS_COCO_dataset_v2/labels/unlabelled_sts/"
label = "STOP"

label_file = "../../../../Downloads/STS_COCO.names"
with open(label_file, "r") as f:
    label2idx = {line.replace("\n", ""):idx for idx, line in enumerate(f.readlines())}
    
ignore_list = []
autocorrect_one_label_based_on_secondary_dataset_folder(path_to_txt, label, label2idx, ignore_list)


Counter({'STOP': 529, 'OTHER': 123, 'garbage': 39, 'truck': 33, '50_SIGN': 8, '80_SIGN': 7, 'traffic light': 4, 'car': 3, '60_SIGN': 2, 'PASS_RIGHT_SIDE': 1, '70_SIGN': 1})


In [5]:
for txt in txts:
    with open(txt, 'r') as f:
        lines = [line.split() for line in f.readlines()]
        for line in lines:
            assert len(line) == 5
            print(line)
        

['21', '0.303516', '0.784896', '0.0585938', '0.0614583']
['21', '0.489062', '0.634896', '0.05625', '0.059375']
['19', '0.300391', '0.628125', '0.0195312', '0.075']
['21', '0.503125', '0.776042', '0.090625', '0.0479167']
['21', '0.232422', '0.784896', '0.0960938', '0.0989583']
['21', '0.283984', '0.7625', '0.0304688', '0.0229167']
['17', '0.241796875', '0.4786458333333333', '0.00703125', '0.015625']
['6', '0.140625', '0.4765625', '0.00625', '0.009375']
['21', '0.135547', '0.495313', '0.0242187', '0.0239583']
['17', '0.241016', '0.478125', '0.00859375', '0.0166667']
['12', '0.14453125', '0.4875', '0.0046875', '0.00625']
['17', '0.24609375', '0.4817708333333333', '0.0078125', '0.017708333333333333']
['21', '0.135547', '0.498958', '0.0242187', '0.025']
['17', '0.246094', '0.481771', '0.0078125', '0.0177083']
['12', '0.144921875', '0.48802083333333335', '0.00546875', '0.007291666666666667']
['6', '0.144921875', '0.4791666666666667', '0.00703125', '0.010416666666666666']
['21', '0.135547', '

['18', '0.5703125', '0.3645833333333333', '0.0765625', '0.10208333333333333']
['17', '0.581641', '0.422917', '0.0179687', '0.03125']
['18', '0.634765625', '0.3458333333333333', '0.08828125', '0.1125']
['17', '0.60703125', '0.41510416666666666', '0.021875', '0.046875']
['17', '0.607422', '0.415104', '0.0226563', '0.046875']
['17', '0.89453125', '0.39010416666666664', '0.0265625', '0.08020833333333334']
['17', '0.895703125', '0.3890625', '0.02890625', '0.06979166666666667']
['17', '0.894531', '0.391146', '0.028125', '0.0802083']
['12', '0.191796875', '0.4942708333333333', '0.00859375', '0.011458333333333333']
['6', '0.19140625', '0.48072916666666665', '0.0109375', '0.015625']
['21', '0.521875', '0.603646', '0.0265625', '0.0260417']
['21', '0.613672', '0.596875', '0.0382813', '0.0229167']
['21', '0.546875', '0.602083', '0.015625', '0.0125']
['17', '0.0199219', '0.492708', '0.0367187', '0.0583333']
['12', '0.21171875', '0.4817708333333333', '0.0109375', '0.015625']
['16', '0.11015625', '0.

['12', '0.615625', '0.32760416666666664', '0.075', '0.096875']
['21', '0.188281', '0.496875', '0.0546875', '0.0583333']
['18', '0.341797', '0.440625', '0.0289063', '0.0395833']
['12', '0.83671875', '0.25104166666666666', '0.0984375', '0.125']
['21', '0.187891', '0.498437', '0.0539063', '0.0552083']
['18', '0.360547', '0.433854', '0.0335938', '0.0427083']
['21', '0.177344', '0.496354', '0.05', '0.053125']
['17', '0.419531', '0.435417', '0.0328125', '0.0458333']
['21', '0.178516', '0.521354', '0.0507812', '0.0510417']
['17', '0.466797', '0.453646', '0.0382813', '0.0510417']
['3', '0.826171875', '0.3692708333333333', '0.02421875', '0.03229166666666667']
['3', '0.448046875', '0.37447916666666664', '0.01953125', '0.030208333333333334']
['3', '0.44765625', '0.37447916666666664', '0.0171875', '0.030208333333333334']
['18', '0.521875', '0.380729', '0.0125', '0.0260417']
['3', '0.821875', '0.37083333333333335', '0.025', '0.03333333333333333']
['3', '0.43515625', '0.3770833333333333', '0.01875',

['1', '0.378125', '0.4088541666666667', '0.028125', '0.036458333333333336']
['11', '0.5234375', '0.42916666666666664', '0.0046875', '0.01875']
['4', '0.13671875', '0.4270833333333333', '0.0046875', '0.00625']
['21', '0.183984', '0.466667', '0.0320312', '0.0395833']
['18', '0.626953', '0.3875', '0.0210938', '0.0291667']
['1', '0.411328125', '0.39947916666666666', '0.03359375', '0.042708333333333334']
['11', '0.549609375', '0.434375', '0.00390625', '0.0125']
['21', '0.185937', '0.469792', '0.0328125', '0.0354167']
['18', '0.677344', '0.380729', '0.0234375', '0.0322917']
['1', '0.517578125', '0.37864583333333335', '0.04921875', '0.065625']
['16', '0.598828125', '0.4317708333333333', '0.01015625', '0.034375']
['18', '0.251171875', '0.4427083333333333', '0.00859375', '0.010416666666666666']
['4', '0.133203125', '0.4427083333333333', '0.00546875', '0.00625']
['21', '0.183594', '0.482292', '0.034375', '0.0354167']
['18', '0.6', '0.430729', '0.0125', '0.0322917']
['18', '0.8125', '0.36875', '0

['12', '0.25546875', '0.5020833333333333', '0.009375', '0.0125']
['17', '0.53359375', '0.45260416666666664', '0.0171875', '0.04479166666666667']
['4', '0.254296875', '0.48854166666666665', '0.01171875', '0.014583333333333334']
['17', '0.53359375', '0.45260416666666664', '0.0171875', '0.042708333333333334']
['17', '0.533594', '0.451042', '0.0171875', '0.04375']
['12', '0.271484375', '0.50625', '0.01015625', '0.0125']
['4', '0.270703125', '0.4901041666666667', '0.01328125', '0.017708333333333333']
['17', '0.75234375', '0.4276041666666667', '0.0203125', '0.07395833333333333']
['17', '0.751953', '0.427083', '0.0195312', '0.0729167']
['12', '0.27578125', '0.50625', '0.0109375', '0.014583333333333334']
['4', '0.275390625', '0.4890625', '0.01484375', '0.017708333333333333']
['17', '0.795703125', '0.4203125', '0.01953125', '0.078125']
['17', '0.795313', '0.421875', '0.0203125', '0.0770833']
['12', '0.279296875', '0.5036458333333333', '0.01171875', '0.015625']
['4', '0.27890625', '0.48697916666

...

['16', '0.052734375', '0.7119791666666667', '0.03828125', '0.053125']
['12', '0.43203125', '0.7625', '0.0171875', '0.022916666666666665']
['21', '0.289453', '0.795833', '0.0570313', '0.0604167']
['17', '0.203906', '0.788542', '0.0234375', '0.01875']
['17', '0.976562', '0.656771', '0.015625', '0.0927083']
['3', '0.744921875', '0.58125', '0.08984375', '0.1125']
['3', '0.335546875', '0.7692708333333333', '0.00546875', '0.007291666666666667']
['21', '0.303516', '0.784375', '0.0585938', '0.0625']
['3', '0.207422', '0.779167', '0.0242187', '0.0229167']
['3', '0.74375', '0.582292', '0.084375', '0.116667']
['3', '0.801953125', '0.5588541666666667', '0.09609375', '0.12604166666666666']
['21', '0.303516', '0.784896', '0.0585938', '0.0614583']
['3', '0.204688', '0.779688', '0.0234375', '0.0239583']
['3', '0.801562', '0.558333', '0.096875', '0.13125']
['21', '0.276953', '0.809375', '0.107031', '0.116667']
['21', '0.354688', '0.793229', '0.034375', '0.0385417']
['19', '0.544531', '0.636458', '0.356

['18', '0.19921875', '0.7666666666666667', '0.021875', '0.029166666666666667']
['21', '0.0527344', '0.892708', '0.102344', '0.110417']
['21', '0.260547', '0.859896', '0.0960938', '0.0510417']
['21', '0.129688', '0.865104', '0.08125', '0.0614583']
['21', '0.457422', '0.841146', '0.0539063', '0.0427083']
['17', '0.335938', '0.80625', '0.015625', '0.025']
['13', '0.358984375', '0.7817708333333333', '0.01640625', '0.021875']
['18', '0.221875', '0.7666666666666667', '0.021875', '0.029166666666666667']
['14', '0.225390625', '0.7947916666666667', '0.02265625', '0.029166666666666667']
['12', '0.103515625', '0.7802083333333333', '0.00859375', '0.010416666666666666']
['21', '0.0585938', '0.895833', '0.114063', '0.108333']
['21', '0.283594', '0.861458', '0.096875', '0.0520833']
['21', '0.480859', '0.841667', '0.0554687', '0.0416667']
['21', '0.148047', '0.866667', '0.0882813', '0.0604167']
['17', '0.359766', '0.80625', '0.0164062', '0.025']
['18', '0.3046875', '0.7619791666666667', '0.0234375', '

['18', '0.674609375', '0.715625', '0.02578125', '0.05625']
['21', '0.489062', '0.848958', '0.120312', '0.14375']
['21', '0.393359', '0.828125', '0.0257812', '0.025']
['21', '0.432422', '0.823438', '0.0179687', '0.0364583']
['21', '0.299609', '0.821354', '0.0226563', '0.0197917']
['28', '0.98125', '0.672396', '0.01875', '0.065625']
['17', '0.674219', '0.714583', '0.025', '0.0520833']
['18', '0.68125', '0.7151041666666667', '0.028125', '0.057291666666666664']
['18', '0.861328125', '0.6229166666666667', '0.04765625', '0.08958333333333333']
['21', '0.488672', '0.852083', '0.119531', '0.141667']
['21', '0.389062', '0.831771', '0.025', '0.0239583']
['18', '0.294531', '0.825', '0.03125', '0.0208333']
['21', '0.430469', '0.826563', '0.021875', '0.0364583']
['18', '0.68125', '0.715104', '0.0265625', '0.0552083']
['18', '0.708203125', '0.70625', '0.03046875', '0.06666666666666667']
['18', '0.965234375', '0.5786458333333333', '0.06953125', '0.109375']
['21', '0.486328', '0.854688', '0.117969', '0

['12', '0.179296875', '0.8078125', '0.03046875', '0.040625']
['12', '0.783984375', '0.7723958333333333', '0.03046875', '0.042708333333333334']
['3', '0.783984375', '0.7270833333333333', '0.03515625', '0.04791666666666667']
['3', '0.176953125', '0.7645833333333333', '0.03359375', '0.04583333333333333']
['21', '0.534766', '0.864062', '0.0882813', '0.090625']
['17', '0.257422', '0.780729', '0.0242187', '0.034375']
['12', '0.257812', '0.780729', '0.025', '0.034375']
['12', '0.178516', '0.808854', '0.0289063', '0.040625']
['12', '0.7890625', '0.7630208333333334', '0.03125', '0.042708333333333334']
['12', '0.166015625', '0.8', '0.03203125', '0.041666666666666664']
['3', '0.163671875', '0.7546875', '0.03515625', '0.046875']
['3', '0.7890625', '0.7166666666666667', '0.0359375', '0.04791666666666667']
['21', '0.532422', '0.856771', '0.0882813', '0.090625']
['12', '0.248828', '0.771875', '0.0242187', '0.0333333']
['12', '0.165625', '0.8', '0.0296875', '0.0395833']
['12', '0.151953125', '0.791145

['21', '0.0859375', '0.843229', '0.084375', '0.0760417']
['21', '0.221484', '0.835938', '0.0445312', '0.0447917']
['21', '0.449609', '0.836979', '0.121094', '0.176042']
['21', '0.369141', '0.821875', '0.0148437', '0.0125']
['28', '0.373437', '0.75', '0.0078125', '0.01875']
['28', '0.383594', '0.749479', '0.0078125', '0.0177083']
['18', '0.325391', '0.732813', '0.0101562', '0.0260417']
['26', '0.275781', '0.827083', '0.0328125', '0.0395833']
['28', '0.312109', '0.733333', '0.0101562', '0.025']
['28', '0.382812', '0.789062', '0.0046875', '0.0177083']
['28', '0.376172', '0.789062', '0.00390625', '0.015625']
['21', '0.275391', '0.827083', '0.0320312', '0.0395833']
['18', '0.121875', '0.797917', '0.015625', '0.0208333']
['16', '0.405078125', '0.7880208333333333', '0.01640625', '0.021875']
['1', '0.976953125', '0.7114583333333333', '0.01484375', '0.03125']
['9', '0.976953125', '0.7526041666666666', '0.01484375', '0.040625']
['21', '0.354297', '0.817708', '0.0304688', '0.0291667']
['28', '0.4

In [130]:
with open("../../../Datasets/Swedish_Traffic_Signs/STS_COCO_dataset_v2_part1/STS_COCO_dataset_v2/labels/unlabelled_sts/1277108134Image000114.txt") as f:
    print(f.readlines())

['13 0.93828125 0.3776041666666667 0.065625 0.08645833333333333\n', '16 0.425 0.6020833333333333 0.00625 0.008333333333333333\n', '16 0.561328125 0.6020833333333333 0.00546875 0.008333333333333333\n', '12 0.9046875 0.15208333333333332 0.00625 0.008333333333333333\n', '21 0.358594 0.619271 0.0453125 0.0447917\n', '\n', '17 0.284375 0.56875 0.0171875 0.0208333\n', '\n', '21 0.448047 0.613021 0.0164062 0.015625\n', '\n', '21 0.210938 0.615625 0.0359375 0.0291667\n', '\n', '21 0.266797 0.617708 0.0335938 0.0229167\n', '\n', '21 0.440234 0.611979 0.0117188 0.015625\n', '\n']


# No training, only visual/manual labeling of bounding boxes

Assuming you have cropped all images

If you have only 10 - 100 samples which you can easily fix by hand, you can move the images directly to the correct class directory. Then run this script.


Let's start by reading the metadata files

In [294]:
# Labelled dataset
batch_name = "batch0"
with open(f"../data/results/{batch_name}_bbox_metadata.txt", 'r') as f:
    lines = f.readlines()
    data = [line.replace("\n", "").split(" ") for line in lines]
    img_bbox_data = [{"filename": row[0], "label": " ".join(row[1:-1]).split("\\")[0], "frame": " ".join(row[1:-1]).split("\\")[1], "bbox_idx": int(row[-1])} for row in data]    
img_bbox_data[:2]

[{'filename': '1277103759Image000001.jpg',
  'label': 'car',
  'frame': 'frame00000.jpg',
  'bbox_idx': 0},
 {'filename': '1277103759Image000006.jpg',
  'label': 'car',
  'frame': 'frame00001.jpg',
  'bbox_idx': 0}]

Now we read the directory structure and the organization of frames (which we have manually moved around)

In [295]:
frame2label = dict()
#path_dataset = "../data/unlabelled_sts2"
path_dataset = "../data/batch0_2"
for label in os.listdir(path_dataset):
    for frame in os.listdir(os.path.join(path_dataset, label)):
        frame2label[frame] = label

Now the question is:
    
How many labels have we manually annotated by visual examination?

In [296]:
num_changed = 0
new_labels = list()
old_labels = list()
for img_bbox in img_bbox_data:
    frame          = img_bbox["frame"]
    original_label = img_bbox["label"]
    
    new_label      = frame2label.get(frame)
    
    if new_label is not None:
        if original_label != new_label:
            new_labels.append(new_label)
            old_labels.append(original_label)
            num_changed += 1
print(f"{num_changed} labels have been moved.\n")

print(f"These are the new labels:")
counts = Counter(new_labels)
print("\n".join([f"{key}  {counts[key]}" for key in counts]))

print()
print(f"This many labels from each has been removed:")
counts = Counter(old_labels)
print("\n".join([f"{key}  {counts[key]}" for key in counts]))
    

1083 labels have been moved.

These are the new labels:
car  57
70_SIGN  28
bus  29
garbage  865
truck  74
80_SIGN  16
STOP  1
OTHER  2
bird  3
bicycle  1
50_SIGN  3
90_SIGN  4

This many labels from each has been removed:
bus  5
70_SIGN  53
100_SIGN  33
car  945
50_SIGN  18
80_SIGN  4
bicycle  1
120_SIGN  23
110_SIGN  1


Now we use a directory to group all bounding boxes/frames for each image

In [297]:
image_frame_dict = dict()
for frame in img_bbox_data:
    filename = frame["filename"]
    original_label = frame["label"]
    frame_name = frame["frame"]
    bbox_idx = frame["bbox_idx"]
    
    
    if image_frame_dict.get(filename) is None:
        image_frame_dict[filename] = list()
    
    image_frame_dict[filename].append((frame_name, original_label, bbox_idx))
image_frame_dict

{'1277103759Image000001.jpg': [('frame00000.jpg', 'car', 0)],
 '1277103759Image000006.jpg': [('frame00001.jpg', 'car', 0)],
 '1277103759Image000011.jpg': [('frame00002.jpg', 'car', 0),
  ('frame00003.jpg', 'car', 1)],
 '1277103759Image000016.jpg': [('frame00004.jpg', 'car', 0)],
 '1277104195Image000003.jpg': [('frame00005.jpg', '100_SIGN', 0),
  ('frame00006.jpg', 'bus', 1)],
 '1277104195Image000008.jpg': [('frame00007.jpg', '100_SIGN', 0),
  ('frame00008.jpg', 'bus', 1)],
 '1277104195Image000013.jpg': [('frame00009.jpg', '100_SIGN', 0)],
 '1277104195Image000018.jpg': [('frame00010.jpg', '100_SIGN', 0),
  ('frame00011.jpg', 'truck', 1)],
 '1277104195Image000023.jpg': [('frame00012.jpg', '100_SIGN', 0),
  ('frame00013.jpg', 'truck', 1)],
 '1277104195Image000028.jpg': [('frame00014.jpg', '100_SIGN', 0),
  ('frame00015.jpg', 'truck', 1)],
 '1277104195Image000033.jpg': [('frame00016.jpg', '100_SIGN', 0),
  ('frame00017.jpg', 'truck', 1),
  ('frame00018.jpg', 'car', 2)],
 '1277104195Image00

In [298]:
# Clean up all '\n'    
def empty_newlines(txt):
    with open(txt, 'r') as f:
        lines = f.readlines()
        a = len(lines)
        lines = [line for line in lines if line != "\n"]
        b = len(lines)
        if a-b != 0:
            with open(txt, 'w') as f:
                for line in lines:
                    f.write(line)

def has_changed(arr1, arr2):
    
    if len(arr1) != len(arr2):
        return True
    
    same = True
    for a, b in zip(arr1, arr2):
        if a != b:
            same = False
    if same:
        return False
    else:
        return True
    
#path_to_txt = "../../../Datasets/Swedish_Traffic_Signs/STS_COCO_dataset_v2_part1/STS_COCO_dataset_v2/labels/unlabelled_sts/"
#path_to_txt = "../../../Datasets/Swedish_Traffic_Signs/STS_COCO_dataset_v2_part1/STS_COCO_dataset_v2/labels/batch1/"
path_to_txt = "../../../Datasets/Swedish_Traffic_Signs/STS_COCO_dataset_v2_part1/STS_COCO_dataset_v2/labels/batch0/"

# Label2index file
label_file = "../../../../Downloads/STS_COCO.names"
with open(label_file, "r") as f:
    label2idx = {line.replace("\n", ""):idx for idx, line in enumerate(f.readlines())}


save_path = "../data/results/new_labels/"
Path(save_path).mkdir(exist_ok=True, parents=True)
new_labels_count = list()

# Make dicts out of data
txt_paths = glob.glob(os.path.join(path_to_txt, "*.txt"))

for txt_path in tqdm(txt_paths):
    
    # Remove all empty '\n' rows
    empty_newlines(txt_path) 
    
    # Get frame data for this specific txt file
    filename = os.path.basename(txt_path).replace(".txt", ".jpg")
    bbox_data = image_frame_dict.get(filename)

    # Read txt file
    with open(txt_path, 'r') as f:
        original_txt_data = f.readlines()
        txt_data = original_txt_data.copy()
    
    if bbox_data is not None:
        for bbox in bbox_data:

            # Original metadata
            frame_name     = bbox[0]
            original_label = bbox[1]
            bbox_idx       = bbox[2]

            # Prediction data
            if frame2label.get(frame_name) is None:

                pass
            else:

                prediction_label = frame2label[frame_name]

                if prediction_label == "garbage":
                    # Throw away this bounding box
                    txt_data[bbox_idx] = "\n"
                    new_labels_count.append(prediction_label)

                elif prediction_label in ignore_list:
                    pass

                else:
                    # Replace the original label with the predicted label
                    txt_string = txt_data[bbox_idx]
                    txt_string = txt_string.split(" ")
                    txt_string[0] =  str(label2idx[prediction_label])
                    txt_string = " ".join(txt_string)
                    txt_data[bbox_idx] = txt_string
                    
                    if prediction_label != original_label:
                        new_labels_count.append(prediction_label)

    if has_changed(txt_data, original_txt_data):
        with open(os.path.join(save_path, os.path.basename(txt_path)), 'w') as f:
            for line in txt_data:
                if line != "\n":

                    if int(line.split(" ")[0]) > 33:
                        print(line)
                    f.write(line)

print(Counter(new_labels_count))
print(len(new_labels_count))


Counter({'garbage': 865, 'truck': 74, 'car': 57, 'bus': 29, '70_SIGN': 28, '80_SIGN': 16, '90_SIGN': 4, 'bird': 3, '50_SIGN': 3, 'OTHER': 2, 'STOP': 1, 'bicycle': 1})
1083


181
